In [1]:
url = 'https://inflationdata.com/Inflation/Consumer_Price_Index/CurrentCPI.asp?reloaded=true'

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import html5lib
import requests
import numpy as np
import os


In [23]:
def scrape(url):
    chromedriver = "/Users/ryanmurray/Downloads/chromedriver" # path to the chromedriver executable
    os.environ["webdriver.chrome.driver"] = chromedriver


    driver = webdriver.Chrome(chromedriver)

    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'lxml')
    driver.quit()
    
    return soup

def parse_table(soup,table="table_name"):
    
    #takes table with "table_name" from scraped website and outputs pandas dataframe
    tab = soup.find("table",{"id": table})
    rows = tab.find_all('tr')
    
    #get header and remove leading and trailing blank strings
    header = rows[0].text.split('\n')

    
    #create list of rows
    rows_list = []
    for row in rows:
        cur_row = []
        for c in row:
            try: cur_row.append(c.text)
            except: cur_row.append('')
        if len(cur_row) == len(header):
            rows_list.append(cur_row)
            
    return pd.DataFrame(rows_list,columns=header)

In [4]:
soup = scrape(url)

In [24]:
inf = parse_table(soup,'table2')

In [29]:
tab = soup.find('table',{'id':'table2'})

In [58]:
rows = tab.find_all('tr')

In [59]:
header = []
for r in rows[0]:
    header.append(r.text)

In [60]:
header

['Year',
 'Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec',
 'Ave']

In [65]:
rows_list = []
for row in rows[1:]:
    cur_row = []
    for d in row:
        cur_row.append(d.text)
    rows_list.append(cur_row)

In [54]:
r2 = []
for r in rows[2]:
    r2.append(r.text)

In [64]:
for r in rows[3]:
    print(r.text)

2016
236.916
237.111
238.132
239.261
240.229
241.018
240.628
240.849
241.428
241.729
241.353
241.432
240.008


In [66]:
rows_list

[['2018',
  '247.867',
  '248.991',
  '249.554',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA',
  'NA'],
 ['2017',
  '242.839',
  '243.603',
  '243.801',
  '244.524',
  '244.733',
  '244.955',
  '244.786',
  '245.519',
  '246.819',
  '246.663',
  '246.669',
  '246.524',
  '245.120'],
 ['2016',
  '236.916',
  '237.111',
  '238.132',
  '239.261',
  '240.229',
  '241.018',
  '240.628',
  '240.849',
  '241.428',
  '241.729',
  '241.353',
  '241.432',
  '240.008'],
 ['2015',
  '233.707',
  '234.722',
  '236.119',
  '236.599',
  '237.805',
  '238.638',
  '238.654',
  '238.316',
  '237.945',
  '237.838',
  '237.336',
  '236.525',
  '237.017'],
 ['2014',
  '233.916',
  '234.781',
  '236.293',
  '237.072',
  '237.900',
  '238.343',
  '238.250',
  '237.852',
  '238.031',
  '237.433',
  '236.151',
  '234.812',
  '236.736'],
 ['2013',
  '230.280',
  '232.166',
  '232.773',
  '232.531',
  '232.945',
  '233.504',
  '233.596',
  '233.877',
  '234.149',
  '233.546',
  '233.069

In [2]:
inflation = pd.DataFrame(rows_list,columns=header).iloc[1:19]

NameError: name 'pd' is not defined

In [84]:
inflation.to_pickle('inflation.pkl')

In [1]:
inflation

NameError: name 'inflation' is not defined

In [86]:
ave = inflation[['Year','Ave']]

In [87]:
cols = ave.columns.tolist()
for col in cols:
    try:
        ave[col] = pd.to_numeric(ave[col])
    except:
        pass

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [88]:
ave

,Year,Ave
1,2017,245.120
2,2016,240.008
3,2015,237.017
4,2014,236.736
5,2013,232.957
6,2012,229.594
7,2011,224.939
8,2010,218.056
9,2009,214.537
10,2008,215.303


In [89]:
ave['factor'] = ave.loc[1,'Ave']/ave['Ave']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
inf_d = ave.drop(['Ave'],axis=1).set_index(['Year']).T.to_dict()

In [102]:
inf_d[2017]['factor']

1.0

In [108]:
new_d = {}
for k in inf_d:
    new_d[k] = inf_d[k]['factor']

In [109]:
new_d

{2017: 1.0,
 2016: 1.021299290023666,
 2015: 1.0341874211554445,
 2014: 1.0354149770208165,
 2013: 1.0522113523096537,
 2012: 1.0676237183898534,
 2011: 1.089717656786951,
 2010: 1.1241149062626115,
 2009: 1.1425534989302544,
 2008: 1.1384885486964882,
 2007: 1.1822013870802828,
 2006: 1.215873015873016,
 2005: 1.2550947260624679,
 2004: 1.297617787188989,
 2003: 1.3324635790389214,
 2002: 1.3626862352679565,
 2001: 1.3843112893206078,
 2000: 1.4234610917537749}

In [110]:
import pickle

file = open('inflation_factor.pkl','wb')

pickle.dump(new_d,file)